In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# split data

In [8]:
import os
import random
from glob import glob
from shutil import rmtree, copyfile

labels = [s.split('/')[1] for s in glob('data/*')]
rmtree('training', ignore_errors=True)
rmtree('validation', ignore_errors=True)
random.seed(1024)
for label in labels:
    jpgs = glob('data/'+label+'/*.jpg')
    random.shuffle(jpgs)
    l = len(jpgs)//5
    os.makedirs('training/'+label)
    for jpg in jpgs[l:]:
        copyfile(jpg, 'training/'+label+'/'+os.path.basename(jpg))
    os.makedirs('validation/'+label)
    for jpg in jpgs[:l]:
        copyfile(jpg, 'validation/'+label+'/'+os.path.basename(jpg))

# basic VGG

In [27]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Lambda, Flatten, Dropout
from keras.layers.convolutional import ZeroPadding2D, Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import image
from keras.optimizers import Adam

In [13]:
VGG_MEAN = np.array([123.68, 116.779, 103.939]).reshape((3, 1, 1))
def preprocessing(img):
    img = img - VGG_MEAN
    return img

model = Sequential()
model.add(Lambda(preprocessing, input_shape=(3, 224, 224), output_shape=(3, 224, 224)))
# Layer 1
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 2
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# Layer 3
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# Layer 4
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# Layer 5
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())

model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))
model.load_weights('vgg16_bn_tf.h5')

model.pop()
for layer in model.layers:
    layer.trainable=False
model.add(Dense(8, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_14 (ZeroPaddi (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 64, 114, 114)      0         
__________

In [29]:
from sklearn.preprocessing import OneHotEncoder
def onehot(x):
    return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [63]:
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, batch_size=4, class_mode='categorical',
                target_size=(224,224)):
    return gen.flow_from_directory(dirname, 
                                   target_size=target_size, 
                                   class_mode=class_mode, 
                                   shuffle=shuffle, 
                                   batch_size=batch_size)
def get_data(path, target_size=(224,224)):
    batches = get_batches(path, shuffle=False, batch_size=1, class_mode=None, target_size=target_size)
    classes = batches.classes
    filenames = batches.filenames
    return np.concatenate([batches.next() for i in range(batches.samples)]), classes, onehot(classes), filenames

In [30]:
training_batches = get_batches('training', batch_size=64)
validation_batches = get_batches('validation', batch_size=64*2, shuffle=False)

Found 3025 images belonging to 8 classes.
Found 752 images belonging to 8 classes.


In [64]:
trn, trn_classes, trn_labels, trn_filenames = get_data('training')
val, val_classes, val_labels, val_filenames = get_data('validation')

Found 3025 images belonging to 8 classes.
Found 752 images belonging to 8 classes.


In [23]:
import bcolz
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

In [46]:
save_array('trn.dat', trn)
save_array('val.dat', val)
save_array('trn_labels.dat', trn_labels)
save_array('val_labels.dat', val_labels)

In [47]:
trn = load_array('trn.dat')
val = load_array('val.dat')
trn_labels = load_array('trn_labels.dat')
val_labels = load_array('val_labels.dat')

In [28]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
model.fit(trn, trn_labels, batch_size=64, epochs=3, validation_data=(val, val_labels))

Train on 3025 samples, validate on 752 samples
Epoch 1/3
3025/3025 [==============================] - 109s - loss: 2.8602 - acc: 0.4707 - val_loss: 1.4336 - val_acc: 0.6609
Epoch 2/3
3025/3025 [==============================] - 64s - loss: 1.6655 - acc: 0.6440 - val_loss: 0.9854 - val_acc: 0.7646
Epoch 3/3
3025/3025 [==============================] - 64s - loss: 1.3035 - acc: 0.7005 - val_loss: 0.7049 - val_acc: 0.7992


In [49]:
model.save_weights('ft1.h5')

## precompute conv output

In [51]:
layer_idx = [index for index, layer in enumerate(model.layers) if type(layer) is Conv2D][-1]
conv_layers, fc_layers = model.layers[:layer_idx+1], model.layers[layer_idx+1:]

In [52]:
conv_model = Sequential(conv_layers)

In [53]:
conv_feat = conv_model.predict(trn)
conv_val_feat = conv_model.predict(val)

In [54]:
save_array('conv_feat.dat', conv_feat)
save_array('conv_val_feat.dat', conv_val_feat)

In [55]:
conv_feat = load_array('conv_feat.dat')
conv_val_feat = load_array('conv_val_feat.dat')

In [56]:
conv_val_feat.shape

(752, 512, 14, 14)

## train model

In [57]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        Dropout(p/4),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(8, activation='softmax')
    ]

bn_model = Sequential(get_bn_layers(0.6))

In [58]:
bn_model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [60]:
bn_model.fit(conv_feat, trn_labels, batch_size=64, epochs=10, validation_data=(conv_val_feat, val_labels))

Train on 3025 samples, validate on 752 samples
Epoch 1/10
3025/3025 [==============================] - 2s - loss: 0.0958 - acc: 0.9765 - val_loss: 0.1500 - val_acc: 0.9641
Epoch 2/10
3025/3025 [==============================] - 2s - loss: 0.0907 - acc: 0.9775 - val_loss: 0.1359 - val_acc: 0.9721
Epoch 3/10
3025/3025 [==============================] - 2s - loss: 0.0635 - acc: 0.9795 - val_loss: 0.1597 - val_acc: 0.9641
Epoch 4/10
3025/3025 [==============================] - 2s - loss: 0.0673 - acc: 0.9798 - val_loss: 0.1600 - val_acc: 0.9628
Epoch 5/10
3025/3025 [==============================] - 2s - loss: 0.0441 - acc: 0.9871 - val_loss: 0.1612 - val_acc: 0.9707
Epoch 6/10
3025/3025 [==============================] - 2s - loss: 0.0365 - acc: 0.9884 - val_loss: 0.1750 - val_acc: 0.9654
Epoch 7/10
3025/3025 [==============================] - 2s - loss: 0.0368 - acc: 0.9904 - val_loss: 0.1664 - val_acc: 0.9747
Epoch 8/10
3025/3025 [==============================] - 2s - loss: 0.0207 - ac

In [61]:
bn_model.optimizer.lr = 1e-4
bn_model.fit(conv_feat, trn_labels, batch_size=64, epochs=10, validation_data=(conv_val_feat, val_labels))

Train on 3025 samples, validate on 752 samples
Epoch 1/10
3025/3025 [==============================] - 2s - loss: 0.0238 - acc: 0.9924 - val_loss: 0.1852 - val_acc: 0.9721
Epoch 2/10
3025/3025 [==============================] - 2s - loss: 0.0208 - acc: 0.9934 - val_loss: 0.1712 - val_acc: 0.9721
Epoch 3/10
3025/3025 [==============================] - 2s - loss: 0.0208 - acc: 0.9947 - val_loss: 0.1564 - val_acc: 0.9707
Epoch 4/10
3025/3025 [==============================] - 2s - loss: 0.0284 - acc: 0.9947 - val_loss: 0.1679 - val_acc: 0.9694
Epoch 5/10
3025/3025 [==============================] - 2s - loss: 0.0328 - acc: 0.9907 - val_loss: 0.1969 - val_acc: 0.9628
Epoch 6/10
3025/3025 [==============================] - 2s - loss: 0.0228 - acc: 0.9940 - val_loss: 0.1624 - val_acc: 0.9707
Epoch 7/10
3025/3025 [==============================] - 2s - loss: 0.0237 - acc: 0.9940 - val_loss: 0.1991 - val_acc: 0.9654
Epoch 8/10
3025/3025 [==============================] - 2s - loss: 0.0135 - ac

In [62]:
bn_model.save_weights('conv_512_6.h5')

In [ ]:
bn_model.load_weights('conv_512_6.h5')

## multi input, add image_size as input

In [80]:
import PIL

In [70]:
sizes = [PIL.Image.open('training/'+f).size for f in trn_filenames]
id2size = list(set(sizes))
size2id = {o:i for i, o in enumerate(id2size)}
{i:sizes.count(i) for i in id2size}

{(1192, 670): 162,
 (1244, 700): 21,
 (1276, 718): 184,
 (1280, 720): 1718,
 (1280, 750): 485,
 (1280, 924): 52,
 (1280, 974): 319,
 (1334, 750): 25,
 (1518, 854): 30,
 (1732, 974): 29}

In [72]:
from keras.utils.np_utils import to_categorical

trn_sizes_orig = to_categorical([size2id[o] for o in sizes], len(id2size))
trn_sizes_orig

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.]])

In [74]:
val_sizes = [PIL.Image.open('validation/'+f).size for f in val_filenames]
val_sizes_orig = to_categorical([size2id[o] for o in val_sizes], len(id2size))
val_sizes_orig

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.]])

In [76]:
# one hot encoding, then normalization
trn_sizes = trn_sizes_orig - trn_sizes_orig.mean(axis=0) / trn_sizes_orig.std(axis=0)
val_sizes = val_sizes_orig - trn_sizes_orig.mean(axis=0) / trn_sizes_orig.std(axis=0)
trn_sizes

array([[-0.1322526 ,  0.5630276 , -0.34334557, ..., -1.14649928,
        -0.25449151, -0.10008344],
       [-0.1322526 , -0.4369724 , -0.34334557, ..., -0.14649928,
        -0.25449151, -0.10008344],
       [-0.1322526 , -0.4369724 , -0.34334557, ..., -0.14649928,
        -0.25449151, -0.10008344],
       ..., 
       [-0.1322526 , -0.4369724 ,  0.65665443, ..., -1.14649928,
        -0.25449151, -0.10008344],
       [-0.1322526 , -0.4369724 , -0.34334557, ..., -0.14649928,
        -0.25449151, -0.10008344],
       [-0.1322526 , -0.4369724 , -0.34334557, ..., -0.14649928,
        -0.25449151, -0.10008344]])

In [82]:
from keras.layers import Input
from keras.layers.merge import concatenate
from keras.models import Model

inp = Input(conv_layers[-1].output_shape[1:])
sz_inp = Input((len(id2size),))
bn_inp = BatchNormalization()(sz_inp)

p = 0.6
x = MaxPooling2D()(inp)
x = BatchNormalization(axis=1)(x)
x = Dropout(p/4)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p/2)(x)
x = concatenate([x,bn_inp])
x = Dense(8, activation='softmax')(x)

model = Model([inp, sz_inp], x)
model.summary()
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 512, 14, 14)   0                                            
____________________________________________________________________________________________________
max_pooling2d_14 (MaxPooling2D)  (None, 512, 7, 7)     0           input_5[0][0]                    
____________________________________________________________________________________________________
batch_normalization_17 (BatchNor (None, 512, 7, 7)     2048        max_pooling2d_14[0][0]           
____________________________________________________________________________________________________
dropout_14 (Dropout)             (None, 512, 7, 7)     0           batch_normalization_17[0][0]     
___________________________________________________________________________________________

In [84]:
model.fit([conv_feat, trn_sizes], trn_labels, batch_size=64, epochs=3, 
             validation_data=([conv_val_feat, val_sizes], val_labels))

Train on 3025 samples, validate on 752 samples
Epoch 1/3
3025/3025 [==============================] - 2s - loss: 1.2144 - acc: 0.6489 - val_loss: 1.3346 - val_acc: 0.7926
Epoch 2/3
3025/3025 [==============================] - 2s - loss: 0.3298 - acc: 0.9048 - val_loss: 0.4542 - val_acc: 0.8963
Epoch 3/3
3025/3025 [==============================] - 2s - loss: 0.1701 - acc: 0.9554 - val_loss: 0.2007 - val_acc: 0.9428


In [86]:
bn_model.fit(conv_feat, trn_labels, batch_size=64, epochs=8, 
             validation_data=(conv_val_feat, val_labels))

Train on 3025 samples, validate on 752 samples
Epoch 1/8
3025/3025 [==============================] - 2s - loss: 0.0168 - acc: 0.9960 - val_loss: 0.2403 - val_acc: 0.9601
Epoch 2/8
3025/3025 [==============================] - 2s - loss: 0.0227 - acc: 0.9934 - val_loss: 0.2152 - val_acc: 0.9668
Epoch 3/8
3025/3025 [==============================] - 2s - loss: 0.0119 - acc: 0.9964 - val_loss: 0.2396 - val_acc: 0.9654
Epoch 4/8
3025/3025 [==============================] - 2s - loss: 0.0166 - acc: 0.9957 - val_loss: 0.2375 - val_acc: 0.9654
Epoch 5/8
3025/3025 [==============================] - 2s - loss: 0.0186 - acc: 0.9954 - val_loss: 0.2127 - val_acc: 0.9721
Epoch 6/8
3025/3025 [==============================] - 2s - loss: 0.0225 - acc: 0.9927 - val_loss: 0.2142 - val_acc: 0.9734
Epoch 7/8
3025/3025 [==============================] - 2s - loss: 0.0140 - acc: 0.9937 - val_loss: 0.2203 - val_acc: 0.9694
Epoch 8/8
3025/3025 [==============================] - 2s - loss: 0.0158 - acc: 0.995

In [87]:
bn_model.optimizer.lr = 1e-4
bn_model.fit(conv_feat, trn_labels, batch_size=64, epochs=8, 
             validation_data=(conv_val_feat, val_labels))

Train on 3025 samples, validate on 752 samples
Epoch 1/8
3025/3025 [==============================] - 2s - loss: 0.0147 - acc: 0.9954 - val_loss: 0.2072 - val_acc: 0.9707
Epoch 2/8
3025/3025 [==============================] - 2s - loss: 0.0213 - acc: 0.9934 - val_loss: 0.2256 - val_acc: 0.9654
Epoch 3/8
3025/3025 [==============================] - 2s - loss: 0.0213 - acc: 0.9944 - val_loss: 0.1591 - val_acc: 0.9774
Epoch 4/8
3025/3025 [==============================] - 2s - loss: 0.0235 - acc: 0.9934 - val_loss: 0.1973 - val_acc: 0.9721
Epoch 5/8
3025/3025 [==============================] - 2s - loss: 0.0178 - acc: 0.9944 - val_loss: 0.1995 - val_acc: 0.9721
Epoch 6/8
3025/3025 [==============================] - 2s - loss: 0.0136 - acc: 0.9970 - val_loss: 0.1619 - val_acc: 0.9747
Epoch 7/8
3025/3025 [==============================] - 2s - loss: 0.0168 - acc: 0.9934 - val_loss: 0.1866 - val_acc: 0.9734
Epoch 8/8
3025/3025 [==============================] - 2s - loss: 0.0095 - acc: 0.996

## multi output, bounding boxes